### Import Libraries and Load Environment Variables

In [8]:
import requests
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Retrieve the API key from environment variables
api_key = os.getenv("GROQCLOUD_API_KEY")

# Define the API URL and headers for GroqCloud API
api_url = "https://api.groq.com/openai/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}


###  Load Negative Areas Function

In [9]:
# Load negative areas from a file
def load_negative_areas(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file.readlines()]


### Process Feedback Function

In [10]:
# Function to process feedback and analyze sentiment
def process_feedback(feedback):
    # Define the task prompt that trains the model using the feedback
    task_prompt = (
        "You are a highly advanced AI model tasked with analyzing customer feedback "
        "to identify areas that caused dissatisfaction. Focus on key aspects of the experience, such as "
        "dining, cleanliness, staff behavior, room amenities, parking facilities, and other service-related "
        "issues. For each piece of feedback, identify whether the sentiment is positive or negative. "
        "If negative, highlight specific aspects mentioned in the feedback that contributed to the dissatisfaction. "
        "Provide a structured response indicating the sentiment and the areas of concern."
    )
        
    # Load the negative areas from the file
    negative_areas = load_negative_areas('negative_areas.txt')  
    # Create the input payload for the API request
    data = {
        "model": "llama-3.3-70b-versatile",
        "messages": [
            {
                "role": "user",
                "content": f"Analyze the sentiment of the following feedback: '{feedback}'. Return a sentiment (Positive or Negative)."
            }
        ]
    }

    # Make a POST request to GroqCloud's API
    response = requests.post(api_url, headers=headers, json=data)

    # Check if the response is successful and process the result
    if response.status_code == 200:
        result = response.json()  # If successful, process the response data
        print("\nAPI Response:", result)  # Debugging step to inspect the actual response structure
        
        # Extract the sentiment from the response
        try:
            sentiment = result['choices'][0]['message']['content'].strip()  # Adjusted indexing
            print(f"Sentiment: {sentiment}")
            
            found_areas = []
            if "negative" in sentiment.lower():
                print("Negative sentiment detected. Analyzing areas that caused negative feedback...")
                
                # Compare feedback with the negative areas
                found_areas = [area for area in negative_areas if area.lower() in feedback.lower()]
                
                if found_areas:
                    print("\nNegative feedback related to:")
                    for area in found_areas:
                        print(f"- {area}")
                else:
                    print("No specific areas of negative feedback identified.")
                
                # Generate an alert for negative sentiment or service issues
                generate_alert(feedback, found_areas)
            
            # Create the dictionary to store results
            feedback_analysis = {
                'sentiment': 'Negative' if "negative" in sentiment.lower() else 'Positive',
                'areas identified': found_areas
            }
            
            # Print the dictionary
            print("\nFeedback Analysis Dictionary:")
            print(feedback_analysis)
            
        except (IndexError, KeyError) as e:
            print("Unexpected API response structure:", e)
    else:
        print(f"Error: {response.status_code}, {response.text}")


### Generate Alert Function

In [11]:
# Function to generate an alert
def generate_alert(feedback, found_areas):
    # Generate a detailed alert including the negative areas
    print(f"\nALERT GENERATED: Immediate attention required for the following feedback:")
    print(f"Feedback: {feedback}")
    if found_areas:
        print("\nIdentified issues in the following areas:")
        for area in found_areas:
            print(f"- {area}")
    else:
        print("No specific areas identified, but negative sentiment detected.")
   


### Main Function

In [12]:
# Main function to get user input and process feedback
def main():
    # Input: Dynamic user feedback
    feedback = input("Please enter your feedback: ")
    
    # Call the function to process feedback
    process_feedback(feedback)


### Run Main Function        

In [13]:
# Run the main function
main()


Please enter your feedback:  I chose this hotel based on its advertised amenities, but it was a complete letdown. The dining experience was terrible; the food was cold, and the staff seemed inattentive. The air conditioning in my room didn’t work properly, and the parking facilities were a nightmare, with no security in place. On top of that, the check-out process was delayed because the system was down, and no one offered an apology or solution. The overall attitude of the staff and the state of the facilities made this a regretful choice.



API Response: {'id': 'chatcmpl-8c0dca8c-1b87-436e-b338-755afb5a3de4', 'object': 'chat.completion', 'created': 1736170348, 'model': 'llama-3.3-70b-versatile', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'The sentiment of the given feedback is: Negative. \n\nThe customer expresses dissatisfaction with multiple aspects of their hotel stay, including the dining experience, room amenities, parking, and check-out process. They use strong negative words like "complete letdown", "terrible", "nightmare", and "regretful choice", which further emphasizes their negative sentiment. There is no positive comment or praise in the feedback, confirming that the overall sentiment is Negative.'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'queue_time': 0.018030809000000002, 'prompt_tokens': 161, 'prompt_time': 0.023195555, 'completion_tokens': 91, 'completion_time': 0.330909091, 'total_tokens': 252, 'total_time': 0.354104646}, 'system_fingerprint': 'fp_fcc3b74982', 'x

In [14]:
main()

Please enter your feedback:  The hotel stay was fantastic! The staff were exceptionally friendly and accommodating. The rooms were clean, spacious, and well-maintained, with all the promised amenities. The dining experience was delightful, offering a wide variety of delicious food. The check-in and check-out process was smooth and hassle-free. Overall, I had a wonderful experience and would highly recommend this hotel to others.



API Response: {'id': 'chatcmpl-41b6e450-a9d0-4dd8-8fc5-cd517d7a67ef', 'object': 'chat.completion', 'created': 1736170426, 'model': 'llama-3.3-70b-versatile', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'The sentiment of the given feedback is: Positive.'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'queue_time': 0.019575843, 'prompt_tokens': 129, 'prompt_time': 0.017574346, 'completion_tokens': 11, 'completion_time': 0.04, 'total_tokens': 140, 'total_time': 0.057574346}, 'system_fingerprint': 'fp_fcc3b74982', 'x_groq': {'id': 'req_01jgxx4krxf48s34q4ymmyz2b9'}}
Sentiment: The sentiment of the given feedback is: Positive.

Feedback Analysis Dictionary:
{'sentiment': 'Positive', 'areas identified': []}
